In [142]:
import numpy as np
import pandas as pd
import random
import time

In [143]:
n=200
#Create empty matrix A
A = np.zeros((n,n))

In [144]:
#Create y
np.random.seed(1)
y = np.random.choice([0,1,2], size=(n,1), p=[0.5,0.3,0.2])

In [145]:
#Create theta
np.random.seed(2)
theta = np.random.uniform(0.0, 1.0, n)

In [146]:
#normalize theta
theta2 = theta/sum(theta)

In [147]:
np.random.seed(3)
for i in range(0,n):
    for j in range(i+1,n):
        if y[i] == y[j]:
            A[i,j]=np.random.binomial(2,theta2[i]*theta2[j]*0.5)
        elif y[i] == 0 and y[j] == 1:
            A[i,j] = np.random.binomial(2, theta2[i]*theta2[j]*0.3)
        elif y[i] == 0 and y[j] == 2:
            A[i,j] =np.random.binomial(2, theta2[i]*theta2[j]*0.1)
        elif  y[i] == 1 and y[j] == 0:
            A[i,j] =np.random.binomial(2, theta2[i]*theta2[j]*0.3)
        elif y[i] == 1 and y[j] == 2:
            A[i,j]=np.random.binomial(2, theta2[i]*theta2[j]*0.3)
        elif y[i] == 2 and y[j] == 0:
            A[i,j] = np.random.binomial(2, theta[i]*theta[j]*0.1)
        elif y[i] == 2 and y[j] == 1:
            A[i,j] =np.random.binomial(2, theta[i]*theta[j]*0.3)

In [148]:
#Enforce Symmetric
for i in range(0,n):
    for j in range(0,n):
        A[j,i]=A[i,j]
A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Split Data

In [149]:
from sklearn.model_selection import RepeatedKFold
rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
for train_index, test_index in rkf.split(A):
     print("TRAIN:", train_index, "TEST:", test_index)
     X_train, X_test = A[train_index], A[test_index]
     y_train, y_test = y[train_index], y[test_index]
     X1 = X_train.transpose()
     X_train, X_test = X1[train_index], X1[test_index]

TRAIN: [  0   1   2   4   5   7   8   9  10  11  12  13  14  15  16  18  19  20
  22  23  27  29  30  31  32  33  35  36  37  38  41  43  44  45  46  47
  48  49  52  57  58  59  60  61  62  63  64  65  66  67  68  69  70  72
  73  74  75  76  78  79  80  81  82  83  85  86  87  88  89  90  91  92
  94  95  96  97  98  99 100 101 102 103 105 106 107 108 109 110 111 112
 113 114 115 118 120 121 123 124 125 126 127 128 129 130 131 132 133 135
 136 137 138 139 140 143 144 145 146 147 150 152 153 155 157 158 159 160
 161 162 163 164 166 167 168 169 170 171 172 173 175 176 177 178 179 181
 182 183 184 185 186 187 189 190 191 192 193 194 196 197 198 199] TEST: [  3   6  17  21  24  25  26  28  34  39  40  42  50  51  53  54  55  56
  71  77  84  93 104 116 117 119 122 134 141 142 148 149 151 154 156 165
 174 180 188 195]
TRAIN: [  0   2   3   4   5   6   7   8   9  11  13  14  16  17  18  20  21  22
  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41
  42  44  47  49  50

In [150]:
X_train.shape,X_test.shape

((160, 160), (40, 160))

# 1

In [151]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
begin = time.time() 
clf = LinearDiscriminantAnalysis()
clf.fit(X_train, y_train.ravel())
LDAacc = clf.score(X_test, y_test)
print(LDAacc)
time.sleep(1) 
end = time.time()
print(f"Total runtime of the program is {end - begin}") 

0.325
Total runtime of the program is 1.0376100540161133


# 2

In [152]:
from scipy import linalg
U, s, VT = linalg.svd(A)
U.shape,  s.shape, VT.shape

((200, 200), (200,), (200, 200))

In [153]:
newx=[]
for c in range(1,10) :
   newx.append(U[:,0:c]*s[0:c]**0.5)

In [154]:
begin1 = time.time()
clf2 = LinearDiscriminantAnalysis()
SVDaccuracy=[]
for i in range(0,8):
    clf.fit(newx[i], y.ravel())
    SVDaccuracy.append(clf.score(newx[i], y.ravel()))
print(SVDaccuracy)
time.sleep(2) 
end1 = time.time()
print(f"Total runtime of the program is {end1 - begin1}")

[0.485, 0.76, 0.765, 0.765, 0.78, 0.78, 0.78, 0.78]
Total runtime of the program is 2.0571298599243164


# 3

In [155]:
w = np.zeros((int(n/5*4),3))
for x in range(0,int(n/5*4)):
    i=int(y_train[x])
    w[x][i]=1

In [156]:
z_train=np.matmul(X_train,w)

In [157]:
z_test = np.matmul(X_test,w)

In [158]:
begin2 = time.time()
from sklearn.neural_network import MLPClassifier
clf3 = MLPClassifier(random_state=2, max_iter=5000)
clf3.fit(z_train, y_train.ravel())
ZYAcc = clf3.score(z_test, y_test)
print(ZYAcc)
time.sleep(3) 
end2 = time.time()
print(f"Total runtime of the program is {end2 - begin2}")

0.75
Total runtime of the program is 4.046205997467041


# 4

In [159]:
begin3 = time.time()
clf4 = LinearDiscriminantAnalysis()
clf4.fit(z_train,y_train.ravel())
ZYLDA = clf4.score(z_test, y_test.ravel())
print(ZYLDA)
time.sleep(4) 
end3 = time.time()
print(f"Total runtime of the program is {end3 - begin3}")

0.8
Total runtime of the program is 4.009154796600342
